cutline filling 
- what: transect are diagonal, and we want to create 64x64 patches for image models
- why: to train image mdoels 
- where: transect available (RNG) or (TLS)
- how: use bbox for the patches > buffer bbox n pixels 

In [1]:
### use magic tools to make into a python script 
import os 
import subprocess
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import box  
from glob import glob

def clip_raster_by_vector_cutline(raster_fn, out_fn, bbox_fn):
    pass 

def clip_raster_by_vector_bbox(raster_fn, out_fn, bbox_fn):

    if not os.path.isfile(out_fn):
        bbox = gpd.read_file(bbox_fn)
        with rasterio.open(raster_fn) as src:
            
            bbox = bbox.to_crs(src.crs)
            raster_bounds = box(*src.bounds)  # Convert bounds to a Shapely Polygon
            raster_gdf = gpd.GeoDataFrame(geometry=[raster_bounds], crs=src.crs)
            
            # Check if bbox overlaps with raster bounds
            if not bbox.intersects(raster_gdf).any():
                raise ValueError(f"No overlap between bbox and raster {raster_fn}")
            
            # Clip raster using bbox geometry
            out_image, out_transform = mask(src, bbox.geometry, crop=True)
            out_meta = src.meta
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })
        
        # Save clipped raster
        with rasterio.open(out_fn, "w", **out_meta) as dest:
            dest.write(out_image)


def reproject_files_seq(tiff_files, reproj_dir,epsgcode):
    #epsgcode = 4326 4979
    os.makedirs(reproj_dir, exist_ok=True)
    reprojected_files = []
    for tiff_file in tiff_files:
        # Construct the output file path for the reprojected file
        base_name = os.path.splitext(os.path.basename(tiff_file))[0]
        reproj_tiff = os.path.join(reproj_dir, f"{base_name}_reproj.tif")
        gdalreproject(fi=tiff_file, fo=reproj_tiff, epsgcode=epsgcode)
        reprojected_files.append(reproj_tiff)
    return reprojected_files

def reproject_files_par():
    pass 

def gdalreproject(fi, fo, res=1/3600, epsgcode=4749):
    """
    #4749 #4326
    Reprojects a raster file using gdalwarp.

    Parameters:
        fi (str): Input file path.
        fo (str): Output file path.
        res (float): Resolution for both x and y (default: 1/3600 degrees).
        epsgcode (int): EPSG code for the target projection (default: 4326).

    Returns:
        None
    """
    xres = yres = res  # Set x and y resolution to the same value by default

    # Check if the output file already exists
    if not os.path.isfile(fo):
        # Construct the gdalwarp command
        cmd_reproject = [
            "gdalwarp",
            "-t_srs", f"EPSG:{epsgcode}",  # Target spatial reference system
            "-tr", str(xres), str(yres),  # Resolution for x and y
            fi,  # Input file
            fo   # Output file
        ]

        try:
            # Run the gdalwarp command
            subprocess.run(cmd_reproject, check=True)
            print(f"Reprojected {fi} -> \n{fo}")
        except subprocess.CalledProcessError as e:
            print(f"Error reprojecting {fi}: {e}")
            print(f"Command attempted: {' '.join(cmd_reproject)}")
    else:
        print(f"Output file {fo} already exists. Skipping reprojection.")

# def list2txt(txt, files):
#     with open(txt, "w") as f:
#         f.write("\n".join(files))

def list2txt(txt, files):
    with open(txt, "w") as f:
        for fi in files:
            f.write(fi+'\n')

def gdalbuildvrt(vrt, txt, epsgcode):
    if not os.path.isfile(vrt):
        cmd_vrt = [
        "gdalbuildvrt",
        "-a_srs", f"EPSG:{epsgcode}",    # Assign the specified EPSG code
        "-input_file_list", txt,         # Input file list (text file)
        vrt                              # Output VRT file
        ]

        try:
            # Run the GDAL command to create the VRT
            subprocess.run(cmd_vrt, check=True)
            print(f"VRT file created successfully: {vrt}")
        except subprocess.CalledProcessError as e:
            print(f"Error creating VRT file: {e}")

1. Clip variables (VRTs) by transect bbox 

In [2]:
pdem_fn= "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/pdem/pdem.vrt"
s1_fn=  "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/s1/s1.vrt"
s2_fn = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/s2/s2.vrt"
esawc_fn = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/esawc/esawc.vrt"
ldem_fn = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/ldem/ldem.vrt"
edem_wgs_fn = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/edem_wgs/edem_wgs.vrt"

cutline_dir = "/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/cutline/"
bbox_dir = "/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/bbox/"
outdir = exp_dir = "/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment/"
reproj_dir = "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj"
tiff_files = glob("/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_OverlapPatchesLiDAR/*.tif")

In [3]:
bbox_files = glob(f"{bbox_dir}/*DTM_NP*.gpkg")
bbox_fn = bbox_files[0]
outdir = exp_dir
bbox_fn

'/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/bbox/DTM_NP_T-0007_EPSG5355.gpkg'

In [4]:
txt = 'transects.txt'
vrt = 'transects.vrt'
epsgcode = 4749#4326
reprojected_files = reproject_files_seq(tiff_files, reproj_dir,epsgcode)
list2txt(txt, files=reprojected_files)
gdalbuildvrt(vrt, txt, epsgcode)

Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0007_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0018_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0019_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0020_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0215_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0384_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0388_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0389_reproj.tif already

In [5]:
def get_bbox_from_gpkg(gpkg_path, layer=None, target_epsg=4749):
    """
    Extracts the bounding box from a GeoPackage file and reprojects it to the target EPSG code.

    Parameters:
        gpkg_path (str): Path to the GeoPackage file.
        layer (str): Name of the layer in the GeoPackage (optional).
        target_epsg (int): EPSG code to which the bbox should be reprojected.

    Returns:
        tuple: Bounding box as (xmin, ymin, xmax, ymax) in the target CRS.
    """
    # Read the GeoPackage file
    gdf = gpd.read_file(gpkg_path, layer=layer)

    # Get the original bounding box
    original_bbox = gdf.total_bounds  # Returns [xmin, ymin, xmax, ymax]
    print(f"Original Bounding Box (EPSG:{gdf.crs.to_epsg()}): {original_bbox}")

    # Reproject the GeoDataFrame to the target CRS
    gdf = gdf.to_crs(epsg=target_epsg)

    # Get the reprojected bounding box
    reprojected_bbox = gdf.total_bounds
    print(f"Reprojected Bounding Box (EPSG:{target_epsg}): {reprojected_bbox}")

    return reprojected_bbox

def gdal_regrid_by_cutline():
    pass 

In [6]:
def get_raster_epsg(raster_path):
    """
    Get the EPSG code of a raster file using rasterio.

    Parameters:
        raster_path (str): Path to the raster file.

    Returns:
        int: EPSG code, or None if CRS is undefined.
    """
    with rasterio.open(raster_path) as src:
        crs = src.crs  # Get the CRS of the raster
        if crs:
            return crs.to_epsg()  # Convert CRS to EPSG code (if possible)
        else:
            return None

def gdal_regrid_by_bbox(fi, fo, bbox, ndv=-9999.,res=1/3600, epsgcode=4749):
    """
        Parameters:
        fi (str): Input file path.
        fo (str): Output file path.
        bbox (tuple): Bounding box as (xmin, ymin, xmax, ymax).
        res (float): Resolution for both x and y (default: 1/3600 degrees).
        epsgcode (int): EPSG code for the target projection (default: 4749).

    Returns:
        None
    """
    xmin, ymin, xmax, ymax = bbox
    xres = yres = res  # Set x and y resolution to the same value by default

    # Check if the output file already exists
    if not os.path.isfile(fo):
        # Construct the gdalwarp command
        repsgcode = get_raster_epsg(fi)
        print(f"{repsgcode}=={epsgcode}?")
        if epsgcode == repsgcode:
            cmd_reproject = ["gdalwarp","-tap",
                             "-tr", str(xres), str(yres),
                             "-te", str(xmin), str(ymin), str(xmax), str(ymax),
                             "-dstnodata", str(ndv),fi, fo ]
        else:
            cmd_reproject = ["gdalwarp","-tap",
                             "-t_srs", f"EPSG:{epsgcode}",  
                             "-te_srs", f"EPSG:{epsgcode}",
                             "-tr", str(xres), str(yres),  
                             "-te", str(xmin), str(ymin), str(xmax), str(ymax),
                             "-dstnodata", str(ndv),fi, fo ]
        try:
            # Run the gdalwarp command
            subprocess.run(cmd_reproject, check=True)
            print(f"Reprojected {fi} -> \n{fo}")
        except subprocess.CalledProcessError as e:
            print(f"Error reprojecting {fi}: {e}")
            print(f"Command attempted: {' '.join(cmd_reproject)}")
    else:
        print(f"Output file {fo} already exists. Skipping reprojection.")

def regrid(fi, fo, bbox, ndv=-9999.,res=1/3600, epsgcode=4749):
    xmin, ymin, xmax, ymax = bbox
    xres = yres = res 
    repsgcode = get_raster_epsg(fi)
    cmd_reproject = [
    "gdalwarp",
    "-tap",
    "-s_srs", f"EPSG:{repsgcode}",
    "-t_srs", f"EPSG:{epsgcode}",
    "-tr", str(xres), str(yres),
    "-te_srs", f"EPSG:{epsgcode}",
    "-te", str(xmin), str(ymin), str(xmax), str(ymax),
    "-dstnodata", str(ndv),
    fi, fo]
    try:
        # Run the gdalwarp command
        subprocess.run(cmd_reproject, check=True)
        print(f"Reprojected {fi} -> \n{fo}")
    except subprocess.CalledProcessError as e:
        print(f"Error reprojecting {fi}: {e}")
        print(f"Command attempted: {' '.join(cmd_reproject)}")

In [7]:

#bbox = get_bbox_from_gpkg(bbox_fn)
#gdal_regrid_by_bbox(fi=vrt, fo=f"{outdir}/clipped_ldem.tif",bbox=bbox)
#gdal_regrid_by_bbox(fi=s1_fn, fo=f"{outdir}/clipped_s1.tif",bbox=bbox)
#gdal_regrid_by_bbox(fi=edem_wgs_fn, fo=f"{outdir}/clipped_edem.tif",bbox=bbox)

In [8]:
clip_raster_by_vector_bbox(s1_fn, f"{outdir}/clipped_s1.tif", bbox_fn)
clip_raster_by_vector_bbox(edem_wgs_fn, f"{outdir}/clipped_edem.tif", bbox_fn)
clip_raster_by_vector_bbox(vrt, f"{outdir}/clipped_ldem.tif", bbox_fn)

In [9]:
# build it step by step

In [10]:
import os
import rasterio
from rasterio.transform import Affine

# Define paths
dtm_path = f"{outdir}/clipped_ldem.tif"  # 1-band DTM reference
dsm_path = f"{outdir}/clipped_edem.tif"  # 1-band DSM
dif_path = f"{outdir}/clipped_zdif.tif"    # Output path for DSM - DTM

if not os.path.exists(dif_path):
    with rasterio.open(dsm_path) as dsm_src, rasterio.open(dtm_path) as dtm_src:
        # Validate that the rasters are compatible
        if dsm_src.shape != dtm_src.shape:
            raise ValueError("Input rasters do not have the same dimensions.")
        if dsm_src.crs != dtm_src.crs:
            raise ValueError("Input rasters do not have the same CRS.")
        # if ra_src.transform != rb_src.transform:
        #     raise ValueError("Input rasters do not have the same transform.")

        # Read the raster data
        dsm = dsm_src.read(1)  # Read the first band of DTM
        dtm = dtm_src.read(1)  # Read the first band of DSM

        # Compute the difference (DSM - DTM)
        dif = dsm - dtm
        ndv = -9999.
        dif[dtm < -99.] = ndv
        dif[dtm > 10000] = ndv

        # Copy metadata from one of the input rasters
        out_meta = dsm_src.meta
        out_meta.update({'nodata':ndv})

    # Write the result to ypath
    with rasterio.open(ypath, "w", **out_meta) as dst:
        dst.write(dif, 1)

    print(f"Computed difference raster saved to {ypath}")
else:
    print(f"Output file {dif_path} already exists. Skipping computation.")
# make it into a function called create_zdiff 
# make it so that the new nodata value works well in soft like QGIS
# and keep only zdiff values for where dtm are valid 

Output file /media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment//clipped_zdif.tif already exists. Skipping computation.


In [11]:
import os
import rasterio
from rasterio.transform import Affine

def create_zdiff(dsm_path, dtm_path, dif_path, ndv=-9999.0):
    """
    Computes the difference (DSM - DTM) and saves it to the specified output path.
    
    Parameters:
        dsm_path (str): Path to the DSM raster file.
        dtm_path (str): Path to the DTM raster file.
        dif_path (str): Path to save the difference raster (DSM - DTM).
        ndv (float): Nodata value for the output raster (default is -9999.0).
        
    Returns:
        None
    """
    # Check if the output file already exists
    if os.path.exists(dif_path):
        print(f"Output file {dif_path} already exists. Skipping computation.")
        return

    # Open the input rasters
    with rasterio.open(dsm_path) as dsm_src, rasterio.open(dtm_path) as dtm_src:
        # Validate that the rasters are compatible
        if dsm_src.shape != dtm_src.shape:
            raise ValueError("Input rasters do not have the same dimensions.")
        if dsm_src.crs != dtm_src.crs:
            raise ValueError("Input rasters do not have the same CRS.")
        if dsm_src.transform != dtm_src.transform:
            raise ValueError("Input rasters do not have the same transform.")

        # Read the raster data
        dsm = dsm_src.read(1)  # Read the first band of DSM
        dtm = dtm_src.read(1)  # Read the first band of DTM

        # Compute the difference (DSM - DTM)
        dif = dsm - dtm

        # Apply nodata mask: retain only valid DTM values
        valid_mask = (dtm > -99.) & (dtm < 10000)  # Define valid DTM range
        dif[~valid_mask] = ndv  # Set invalid areas to nodata

        # Copy metadata from one of the input rasters and update nodata value
        out_meta = dsm_src.meta
        out_meta.update({
            'nodata': ndv,
            'dtype': 'float32'  # Ensure output dtype is float32 for better GIS compatibility
        })

    # Write the result to the output path
    with rasterio.open(dif_path, "w", **out_meta) as dst:
        dst.write(dif.astype('float32'), 1)

    print(f"Computed difference raster saved to {dif_path}")

dsm_path = f"{outdir}/clipped_edem.tif"
dtm_path = f"{outdir}/clipped_ldem.tif"
dif_path = f"{outdir}/clipped_zdif.tif"

create_zdiff(dsm_path, dtm_path, dif_path)

Output file /media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment//clipped_zdif.tif already exists. Skipping computation.


why chm (dsm-dtm) is lower than dtm ? should it it be higher than dtm and lower than dsm? so what's up with that? make it make sense because it not supposed to physically. context I am wrking with rasters

In [12]:
dsm_path = f"{outdir}/clipped_edem.tif" # feature  1band
dtm_path = f"{outdir}/clipped_ldem.tif" # reference 1band
dif_path = f"{outdir}/clipped_zdif.tif" # target 1band
s1_path = f"{outdir}/clipped_s1.tif" # feature 2bands 
mlpath = f"{outdir}/clipped_tabml.tif"

# read dif_path as taget, create mask of vaid to invalid pixls based on nvd and anything below -99 and above 5000
# read dsm_path and s1_path as feature and filter out the by the mask above 

# train feature with target on valid pixels : use catboost regressor iterations 2000
# predict on all the feature ()
# do dsm - pred and write the output to tif with dsm propertoes to mlpath




In [13]:
import os
import numpy as np
import rasterio
from skimage.transform import resize
from catboost import CatBoostRegressor

def read_raster(file_path):
    """Reads a raster file and returns the array and metadata."""
    with rasterio.open(file_path) as src:
        array = src.read(1)
        profile = src.profile
        return array, profile

def resize_raster(src_array, target_shape):
    """Resizes a raster array to match the target shape using interpolation."""
    resized_array = resize(
        src_array, 
        target_shape, 
        order=1,  # Bilinear interpolation
        mode='constant', 
        anti_aliasing=True
    )
    return resized_array

def preprocess_data(dif_path, dsm_path, s1_path):
    """Reads and processes input raster data."""
    dif, profile = read_raster(dif_path)
    dsm, _ = read_raster(dsm_path)
    
    # Read Sentinel-1 bands
    with rasterio.open(s1_path) as src:
        s1_band1 = src.read(1)
        s1_band2 = src.read(2) if src.count > 1 else src.read(1)  # Handle single-band files
    
    # Resize Sentinel-1 bands to match DSM dimensions
    target_shape = dsm.shape
    s1_band1_resized = resize_raster(s1_band1, target_shape)
    s1_band2_resized = resize_raster(s1_band2, target_shape)
    
    # Create a valid mask based on the difference raster
    valid_mask = (dif > -99) & (dif < 5000)
    
    # Extract valid values
    valid_dsm = dsm[valid_mask]
    valid_s1_band1 = s1_band1_resized[valid_mask]
    valid_s1_band2 = s1_band2_resized[valid_mask]
    valid_dif = dif[valid_mask]
    
    # Combine features
    X_train = np.column_stack((valid_dsm, valid_s1_band1, valid_s1_band2))
    y_train = valid_dif
    
    return X_train, y_train, dsm, s1_band1_resized, s1_band2_resized, profile

def train_model(X_train, y_train):
    """Trains a CatBoost model."""
    model = CatBoostRegressor(iterations=2000, verbose=100)
    model.fit(X_train, y_train)
    return model

def predict_model(model, dsm, s1_band1, s1_band2):
    """Generates predictions for the entire raster grid."""
    all_features = np.column_stack((dsm.flatten(), s1_band1.flatten(), s1_band2.flatten()))
    predictions = model.predict(all_features).reshape(dsm.shape)
    return predictions

def save_raster(output_path, array, profile):
    """Saves a raster file with the specified profile."""
    profile.update(dtype=rasterio.float32, nodata=-9999.0)
    with rasterio.open(output_path, "w", **profile) as dst:
        dst.write(array.astype('float32'), 1)

def transect_augment_data(dif_path, dsm_path, s1_path, output_path):
    """Main function to process rasters, train a model, and generate predictions."""
    # Step 1: Preprocess data
    X_train, y_train, dsm, s1_band1_resized, s1_band2_resized, profile = preprocess_data(dif_path, dsm_path, s1_path)

    # Step 2: Train model
    model = train_model(X_train, y_train)

    # Step 3: Predict and save results
    predictions = predict_model(model, dsm, s1_band1_resized, s1_band2_resized)
    dsm_pred_diff = dsm - predictions
    save_raster(output_path, dsm_pred_diff, profile)
    print(f"Prediction results saved to {output_path}")

    # add the dowscaling ml here @ do the saving models latter 


transect_augment_data(dif_path, dsm_path, s1_path, mlpath)

Learning rate set to 0.03303
0:	learn: 4.5602570	total: 50.9ms	remaining: 1m 41s
100:	learn: 1.8505793	total: 289ms	remaining: 5.44s
200:	learn: 1.7515871	total: 501ms	remaining: 4.48s
300:	learn: 1.7110827	total: 710ms	remaining: 4s
400:	learn: 1.6817320	total: 918ms	remaining: 3.66s
500:	learn: 1.6587208	total: 1.13s	remaining: 3.37s
600:	learn: 1.6398366	total: 1.34s	remaining: 3.11s
700:	learn: 1.6215031	total: 1.55s	remaining: 2.87s
800:	learn: 1.6034715	total: 1.76s	remaining: 2.63s
900:	learn: 1.5883158	total: 1.97s	remaining: 2.4s
1000:	learn: 1.5732051	total: 2.19s	remaining: 2.19s
1100:	learn: 1.5583849	total: 2.4s	remaining: 1.96s
1200:	learn: 1.5444019	total: 2.61s	remaining: 1.74s
1300:	learn: 1.5305429	total: 2.82s	remaining: 1.51s
1400:	learn: 1.5168516	total: 3.03s	remaining: 1.29s
1500:	learn: 1.5033568	total: 3.24s	remaining: 1.08s
1600:	learn: 1.4894817	total: 3.44s	remaining: 859ms
1700:	learn: 1.4773276	total: 3.66s	remaining: 643ms
1800:	learn: 1.4650220	total: 3.

In [ ]:
import sys 
from uvars import topoxcale_dir 
sys.path.append(topoxcale_dir)
from topoxcale.mlxcale import mldownxcale
from topoxcale.sagaxcale import gwrdownxcale

#4749 or  4979
num_rounds = 2000
n = 0  # Set to 0 to use all data
sfix2 = "GWR"
sfix1 = "MLdx"
model_params = {'num_rounds': num_rounds,'verbose': 200}
x_path = dsm_path
y_path = mlpath
model_name = y_path.replace('tif', f'_{num_rounds}.pkl')
out_path = y_path.replace('.tif', f'_{sfix1}.tif')
out_path2 = y_path.replace('.tif', f'_{sfix2}.tif')
#mldownxcale(x_path, y_path, model_name, model_params, out_path, n=0): fix the error
gwrdownxcale(xpath=dsm_path, ypath=mlpath, opath=out_path2,oaux=False,epsg_code=4749, clean=False) 


In [17]:
# gwr downscaling much smoother and better for now 

In [123]:
dsm_path

'/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment//clipped_edem.tif'

In [18]:
# do the weighted mdoels with the paper [x]
# do the dwx - just grab the codes from the other project [x]
# do the todo list to clear up stuff 
# leave this stuff running 
# load into Onedrive and write data augmentation A, and then do B(increment by gridsize)

### do incremental learning for this stuff https://catboost.ai/docs/en/concepts/python-reference_catboostregressor_fit
## predict with gmodel, and check error
## train lmodel, and check the error
## pred with gmodel and check the error 
## pick the best of the two base on residual map (cout wins)